# ML-Based Demand Forecasting for Multi-Warehouse Inventory Optimization

In [1]:
import pandas as pd

def build_features(df, lags=[1,7,14]):
    df = df.copy()

    for lag in lags:
        df[f"lag_{lag}"] = df["daily_demand"].shift(lag)

    df["rolling_mean_7"] = df["daily_demand"].rolling(7).mean()
    df["rolling_std_7"] = df["daily_demand"].rolling(7).std()
    df["day_of_week"] = df["day"] % 7

    df = df.dropna()
    return df

## Demand Forecasting Model

In [2]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np

def train_forecast_model(df):
    X = df.drop(columns=["daily_demand", "warehouse"])
    y = df["daily_demand"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, shuffle=False, test_size=0.2
    )

    model = XGBRegressor(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        random_state=42
    )

    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)

    return model, mae

## Forecast Mean + Uncertainty

In [3]:
def forecast_with_uncertainty(model, X):
    preds = model.predict(X)
    residuals = X["lag_1"] - preds
    sigma = np.std(residuals)

    return preds.mean(), sigma

## ML-Driven Reorder Point

In [4]:
from scipy.stats import norm
import numpy as np

def ml_reorder_point(mu, sigma, lead_time, service_level=0.95):
    z = norm.ppf(service_level)
    return mu * lead_time + z * sigma * np.sqrt(lead_time)

## Full ML-Powered Multi-Warehouse Optimizer

In [6]:
from pandas import DataFrame
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm
import numpy as np

# Assuming build_features, train_forecast_model, forecast_with_uncertainty,
# ml_reorder_point, eoq, and simulate_inventory are defined in the global scope
# (e.g., in previous cells or will be defined subsequently).
# Original imports from src were causing ModuleNotFoundError as 'src' was not found.

def optimize_ml_inventory(df: DataFrame):
    results = []

    for wh in df["warehouse"].unique():
        wdf = df[df["warehouse"] == wh].sort_values("day")

        # Use the globally defined build_features
        features_df = build_features(wdf)

        # Use the globally defined train_forecast_model
        model, mae = train_forecast_model(features_df)

        X_latest = features_df.drop(
            columns=["daily_demand", "warehouse"]
        ).tail(30)

        # Use the globally defined forecast_with_uncertainty
        mu, sigma = forecast_with_uncertainty(model, X_latest)
        lead_time = wdf["lead_time"].iloc[0]

        annual_demand = mu * 365
        # Use the globally defined eoq (assuming it's defined elsewhere)
        Q = eoq(annual_demand, ordering_cost=300, holding_cost=2)
        # Use the globally defined ml_reorder_point
        ROP = ml_reorder_point(mu, sigma, lead_time)

        # Use the globally defined simulate_inventory (assuming it's defined elsewhere)
        sim = simulate_inventory(wdf["daily_demand"], Q, ROP)

        results.append({
            "warehouse": wh,
            "forecast_mae": round(mae, 2),
            "EOQ": round(Q),
            "ROP": round(ROP),
            "stockouts": sim["total_stockouts"],
            "orders": sim["orders_placed"]
        })

    return results

In [11]:
import pandas as pd

df = pd.read_csv("/content/multi_warehouse_demand.csv")

results = optimize_ml_inventory(df)
for r in results:
    print(r)

{'warehouse': 'WH_A', 'forecast_mae': 6.26, 'EOQ': 2527, 'ROP': 265, 'stockouts': np.float32(1344.6289), 'orders': 9}
{'warehouse': 'WH_B', 'forecast_mae': 5.31, 'EOQ': 2051, 'ROP': 259, 'stockouts': np.float32(341.12012), 'orders': 8}
{'warehouse': 'WH_C', 'forecast_mae': 3.48, 'EOQ': 1495, 'ROP': 216, 'stockouts': 119, 'orders': 5}


In [12]:
def eoq(annual_demand, ordering_cost, holding_cost):
    return np.sqrt((2 * annual_demand * ordering_cost) / holding_cost)

In [13]:
def simulate_inventory(daily_demand_series, Q, ROP):
    inventory_level = 0
    orders_placed = 0
    total_stockouts = 0
    inventory_on_order = 0
    lead_time = 7 # Assuming a constant lead time for simplicity in simulation
    order_arrival_day = -1

    for day, demand in enumerate(daily_demand_series):
        # Check for order arrival
        if day == order_arrival_day:
            inventory_level += Q
            inventory_on_order = 0

        # Process demand
        inventory_level -= demand

        # Check for stockout
        if inventory_level < 0:
            total_stockouts += abs(inventory_level)
            inventory_level = 0 # Inventory cannot be negative

        # Check reorder point
        if inventory_level <= ROP and inventory_on_order == 0:
            orders_placed += 1
            inventory_on_order = Q # Assume order is for EOQ quantity
            order_arrival_day = day + lead_time

    return {"total_stockouts": total_stockouts, "orders_placed": orders_placed}